# Malaysia Covid 19
### Live bokeh app of this is at http://mycovid19.my.to:30080

In [1]:
# import needed package
import pandas as pd
import numpy as np
# import holoviews as hv
# import panel as pn
from functools import partial
# from bokeh.models import Range1d, LinearAxis, HoverTool, NumeralTickFormatter, DatetimeTickFormatter
# from bokeh.layouts import layout
from bokeh.io import show, curdoc, output_notebook
# from holoviews import opts
# import panel.widgets as pnw
# hv.extension('bokeh', logo=False)
output_notebook()

Loading BokehJS ...

In [2]:
# import bokeh plotting modules
from bokeh.plotting import figure
from bokeh.models import (ColumnDataSource, ResetTool, SaveTool, 
                          HoverTool, Panel, Label, BasicTickFormatter, 
                          NumeralTickFormatter, LinearAxis, TapTool,
                          DatetimeTickFormatter, Range1d, LabelSet, 
                          Title, Slider, SingleIntervalTicker, FuncTickFormatter, FixedTicker)
from bokeh.models.widgets import (TableColumn, DataTable, NumberFormatter, 
                                  StringFormatter, Div, DateFormatter, 
                                  SelectEditor, Button, RadioButtonGroup, 
                                  Tabs, Select)
from bokeh.layouts import layout, row, column, Spacer, WidgetBox
from bokeh.palettes import brewer, Set1, Category20_20, Category20b_20
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.transform import dodge, stack
from bokeh.core.properties import value
from pyproj import Proj, transform 
from functools import partial
from bokeh.events import Tap

In [3]:
# reference parameter
cur_datetime = pd.to_datetime('today')
past_90_days = (cur_datetime + pd.DateOffset(days=-90)).normalize()


In [4]:
# MOH path base
url_moh = 'https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/'

# citf path base
url_citf = 'https://raw.githubusercontent.com/CITF-Malaysia/citf-public/main/'

# mhlearn path base
url_mhlearn = 'https://raw.githubusercontent.com/mhlearn/covid19/master/'

# import Malaysia major MCO and holidays list
# mco_list = pd.read_excel(url_mhlearn + 'static/date_malaysia_mco_nationwide_holiday.xlsx', 
#                          sheet_name='list', engine='openpyxl')

# hv.Table(mco_list).opts(height=150)

In [5]:
def style_naked(plot):
    '''plot nude chart, without x-axis, y-axis, and its
    label'''
        
    # hide tools
    plot.toolbar.active_drag = None
    plot.toolbar.active_scroll = None
    plot.toolbar.active_tap = None
    plot.toolbar.logo = None
#     plot.toolbar.tools = 
    plot.toolbar_location = None
    plot.toolbar.autohide = True

    # axis layout
    plot.xaxis.major_tick_line_color = None
    plot.xaxis.minor_tick_line_color = None
    plot.xgrid.grid_line_color = None
    plot.xaxis.major_label_text_font_size = '0px'
    plot.xaxis.axis_line_color = None
    plot.yaxis.major_tick_line_color = None
    plot.yaxis.minor_tick_line_color = None
    plot.ygrid.grid_line_color = None
    plot.yaxis.axis_line_color = None
    plot.yaxis.major_label_text_font_size = '0px' ###->change to '0px' to have no label
    plot.outline_line_color = None

    # title layout
#     plot.title.align = "center"
#     plot.title.text_color = "black"
#     plot.title.text_font_size = "11px"
#     plot.title.text_font = 'serif'

    # legend label
    plot.legend.visible = True
#     plot.legend.glyph_height = 0
#     plot.legend.glyph_width = 0
#     plot.legend.label_height = 0
    plot.legend.location = "top_left"
    plot.legend.orientation = "horizontal"
#     plot.legend.border_line_color = True
    plot.legend.background_fill_alpha = 0
    plot.legend.label_text_font_size ="12px"
    plot.legend.click_policy="mute"
    
    return plot


In [6]:
# get all relavent df. logically, sum of daily of all states
# is equal to total daily number in Malaysia csv. But in cases...

# import mysejahtera check-in data
checkin_my = pd.read_csv(url_moh + 'mysejahtera/checkin_malaysia.csv', 
                         parse_dates = ['date'])
checkin_my['state'] = "Malaysia"
checkin_state = pd.read_csv(url_moh + 'mysejahtera/checkin_state.csv', 
                            parse_dates = ['date'])

checkin_df = checkin_my.append(checkin_state, ignore_index=True).sort_values('date')

# import death case data
death_my = pd.read_csv(url_moh + 'epidemic/deaths_malaysia.csv', 
                       parse_dates=['date'])
death_my['state']='Malaysia'
death_state = pd.read_csv(url_moh + 'epidemic/deaths_state.csv', 
                          parse_dates=['date'])

death_df = death_my.append(death_state, ignore_index=True).sort_values('date')

# import icu bed data
icu_state = pd.read_csv(url_moh + 'epidemic/icu.csv', 
                        parse_dates=['date'])
# icu sum by daily,  Malaysia
icu_my = icu_state.resample('D', on='date').sum().reset_index().assign(state='Malaysia')
icu_df = icu_my.append(icu_state).sort_values('date')

# import confirmed/positive case data
cases_my = pd.read_csv(url_moh + 'epidemic/cases_malaysia.csv', 
                       parse_dates=['date'])
cases_my['state']='Malaysia'
cases_state = pd.read_csv(url_moh + 'epidemic/cases_state.csv', 
                          parse_dates=['date'])
cases_df = cases_my.loc[:,['date', 'state','cases_new']].append(cases_state, ignore_index=True).sort_values('date')

# import screening case data
screening_df = pd.read_csv(url_moh + 'epidemic/tests_malaysia.csv', 
                           parse_dates=['date'])

# import cluster data
cluster_df = pd.read_csv(url_moh + 'epidemic/clusters.csv', 
                         parse_dates=['date_announced', 'date_last_onset'])

# import vaccination data
vax_my = pd.read_csv(url_citf + 'vaccination/vax_malaysia.csv', 
                     parse_dates=['date'])
vax_my['state']='Malaysia'
vax_state = pd.read_csv(url_citf + 'vaccination/vax_state.csv', 
                        parse_dates=['date'])

vax_df = vax_my.append(vax_state, ignore_index=True).sort_values('date')

# vaccination registration data
reg_vax_my = pd.read_csv(url_citf + 'registration/vaxreg_malaysia.csv', 
                         parse_dates=['date'])
reg_vax_state = pd.read_csv(url_citf + 'registration/vaxreg_state.csv', 
                            parse_dates=['date'])

reg_vax_df = reg_vax_my.append(reg_vax_state, ignore_index=True).sort_values('date')

# import population data
pop_df = pd.read_csv(url_citf + 'static/population.csv')
pop_df['pop_below_18'] = pop_df['pop'] - pop_df['pop_18']
pop_df['pop_18_60'] = pop_df['pop_18'] - pop_df['pop_60']
pop_df.rename(columns={'pop_60':'pop_60_above'}, inplace=True)
pop_df.rename(columns={'pop_18':'pop_18_above'}, inplace=True)

# merge latest vax registration figure to population
latest_reg_vax = reg_vax_df.set_index('date')\
                           .last('1D')\
                           .reset_index()\
                           .loc[:,['state','total']]\
                           .rename(columns={'total':'total_reg_vax'})
pop_df = pop_df.merge(latest_reg_vax, on='state', how='outer')

# merge latest vaccination number to population
latest_vax = vax_df.set_index('date')\
                   .last('1D')\
                   .loc[:,['state','dose1_cumul', 'dose2_cumul']]\
                   .reset_index(drop=True)
latest_vax['dose1_only_cumul'] = latest_vax['dose1_cumul']-latest_vax['dose2_cumul']
latest_vax.rename(columns={'dose1_cumul':'at_least_dose1'}, inplace=True)
pop_df = pop_df.merge(latest_vax, on='state', how='outer')

# merge population to vax df
vax_df = vax_df.merge(pop_df[['state', 'pop','pop_18_above']], on='state', how='outer').sort_values('date')

# merge screening data to cases df
screening_df['state'] = 'Malaysia'
cases_df = cases_df.merge(screening_df, on=['date','state'], how='outer')

In [7]:
# cluster_df_active = cluster_df[cluster_df['status']=='active']
# cluster_df_active.groupby('category')['state']\
#                  .count()\
#                  .to_frame()\
#                  .merge(cluster_df_active.groupby('category').agg(sum),
#                         left_index=True,
#                         right_index=True)


In [8]:
def vax_plot(df_vax, df_checkin):
    '''plot accum. dose 1 and dose 2 vax status and 
    mysejahtera unique_ind daily checkin'''
    
    # prepare vax rate of dose 1 and dose 2
    df_vax['dose1_rate_adult'] = df_vax['dose1_cumul']/df_vax['pop_18_above']
    df_vax['dose2_rate_adult'] = df_vax['dose2_cumul']/df_vax['pop_18_above']
    df_vax['dose1_rate_pop'] = df_vax['dose1_cumul']/df_vax['pop']
    df_vax['dose2_rate_pop'] = df_vax['dose2_cumul']/df_vax['pop']
    
    # state population
    total_pop = df_vax.set_index('date').last('1D').loc[:,'pop'].values
    
    # prepare active individual rate of mysejhatera checkin
    df_checkin['active_ind_rate'] = df_checkin['unique_ind']/total_pop
    df_checkin['avg_premise_ind'] = df_checkin['checkins']/df_checkin['unique_ind']
    
    # prepare x-axis lable 
    df_vax["x_axis_label"] = df_vax.date.apply(lambda x:x.strftime('%Y, %b-%d'))
    df_vax = df_vax.sort_values("date", ascending=True) 
    df_checkin["x_axis_label"] = df_checkin.date.apply(lambda x:x.strftime('%Y, %b-%d'))
    df_checkin = df_checkin.sort_values("date", ascending=True) 
    
    # x axis label
    x_range_label = df_vax.x_axis_label.to_list() if len(df_vax)>=len(df_checkin) else df_checkin.x_axis_label.to_list()
    
    # hover tool
    checkin_tip = [('date','@date{%Y, %b-%d}'), 
                   ('individual checkin', '@unique_ind{0,0}'),
                   ('active ind. rate over pop.', '@active_ind_rate{0.00%}'),
                   ('avg. premises visit per ind.', '@avg_premise_ind{0.00}')
                  ]
                      
    
    dose1_tip = [('date','@date{%Y, %b-%d}'), 
                 ('taken 1st jab', '@dose1_cumul{0,0}'),
                 ('% over adult pop.', '@dose1_rate_adult{0.00%}'),
                 ('% over pop.', '@dose1_rate_pop{0.00%}')
                ]

    dose2_tip = [('date','@date{%Y, %b-%d}'), 
                 ('taken 2nd jab', '@dose2_cumul{0,0}'),
                 ('% over adult pop.', '@dose2_rate_adult{0.00%}'),
                 ('% over pop.', '@dose2_rate_pop{0.00%}')
                ]
    
    # start figure
    p = figure(x_range=x_range_label, plot_width=600, plot_height=300)
    
    p_checkin = p.vbar(source=df_checkin, 
                       x='x_axis_label', top='unique_ind', 
                       color='#87cefa', legend_label='MySejahtera checkin ind.', 
                       width=0.5, muted_alpha=0.2)
    
    p_vax_dose1 = p.line(source=df_vax, 
                         x='x_axis_label', y='dose1_cumul', 
                         color='green', legend_label='dose 1', 
                         line_width=2, muted_alpha=0.2)
    p_vax_dose2 = p.line(source=df_vax, 
                         x='x_axis_label', y='dose2_cumul', 
                         color='#013220', legend_label='dose 2', 
                         line_width=2, muted_alpha=0.2)
    
    # add hover tools
    p.add_tools(HoverTool(renderers=[p_vax_dose1], 
                          tooltips=dose1_tip,
                          mode="vline",
                          attachment='right',
                          muted_policy = 'ignore',
                          formatters={'@date':'datetime'}
                         )
               )
    
    p.add_tools(HoverTool(renderers=[p_vax_dose2], 
                          tooltips=dose2_tip,
                          mode="vline",
                          attachment='left',
                          muted_policy = 'ignore',
                          formatters={'@date':'datetime'}
                         )
               )
    
    p.add_tools(HoverTool(renderers=[p_checkin], 
                          tooltips=checkin_tip,
                          mode="mouse",
                          muted_policy = 'ignore',
                          formatters={'@date':'datetime'}
                         )
               )
    
    p = style_naked(p)
    
    return p


In [9]:
def case_plot(df_cases):
    '''plot positive case and screening details'''
    
    # prepare extra info
    df_cases['test_sum'] = df_cases[['rtk-ag','pcr']].sum(axis=1)
    df_cases['x_range_label'] = df_cases.date.apply(lambda x:x.strftime('%Y, %b-%d'))
    df_cases['positive_rate'] = df_cases['cases_new']/df_cases['test_sum']
    
    # x range label
    x_range_label = df_cases.x_range_label.to_list()
    y2_range_max = df_cases.positive_rate[np.isfinite(df_cases.positive_rate)].max()
    
    # hover tips
    case_tip = [('date', '@date{%Y, %b-%d}'), 
                ('positive case', '@cases_new{0,0}'), 
                ("rtk-ag test", '@{rtk-ag}{0,0}'), 
                ('pcr test', '@pcr{0,0}')
               ]
    
    rate_tip = [('date', '@date{%Y, %b-%d}'),
                ('positive case rate', '@positive_rate{0.00%}'), 
                ('total test', '@test_sum{0,0}')
               ]
    
    
    # initiate figure
    p = figure(plot_width=600, plot_height=300, x_range=x_range_label)
    
    # positive case plot
    p_case = p.vbar(source=df_cases, x='x_range_label', top='cases_new', 
                    width=0.5, color='red', legend_label='positive case', 
                    muted_alpha=0.2)
        
    # secondary y-axis for traffic frequency on right side
    p.extra_y_ranges = {"case_rate": Range1d(start=0, end=y2_range_max)}
    p.add_layout(LinearAxis(y_range_name="case_rate"), 'right')
    
    # positive case rate plot
    p_rate = p.line(source=df_cases, x='x_range_label', y='positive_rate', 
                    y_range_name='case_rate', line_width=1.5, color = 'blue',
                    legend_label='case ratio', muted_alpha=0.2)  
    
    
    # add hoer tool
    p.add_tools(HoverTool(renderers=[p_case], 
                          tooltips=case_tip,
                          mode="vline",
                          attachment='left',
                          muted_policy='ignore',
                          formatters={'@date':'datetime'}
                         )
               )
    
    p.add_tools(HoverTool(renderers=[p_rate], 
                          tooltips=rate_tip,
                          mode="vline",
                          attachment='right',
                          muted_policy = 'ignore',
                          formatters={'@date':'datetime'}
                         )
               )
    
    # plot style
    p = style_naked(p)
    
    return p
    
    

In [10]:
def icu_plot(df_icu):
    '''plot usage of icu bed and ventilator'''
    
    # prepare extra info
    df_icu['x_range_label'] = df_icu.date.apply(lambda x:x.strftime('%Y, %b-%d'))    
    
    # x range label
    x_range_label = df_icu.x_range_label.to_list()
    
    
    # hover tips
    bed_tip = [('date', '@date{%Y, %b-%d}'), 
               ('total icu bed', '@beds_icu_total{0,0}'), 
              ]
    
    icu_tip = [('date', '@date{%Y, %b-%d}'),
               ('patient in icu', '@icu_covid{0,0}'), 
              ]
    
    vent_tip = [('date', '@date{%Y, %b-%d}'),
                ('patient with vent.', '@vent_covid{0,0}'), 
               ]
    
    
    # initiate figure
    p = figure(plot_width=600, plot_height=300, x_range=x_range_label)
    
    # total available icu bed
    p_bed = p.vbar(source=df_icu, x='x_range_label', top='beds_icu_total', 
                   width=0.5, color='green', legend_label='total icu bed', 
                   muted_alpha=0.2)
           
    # occupied icu / patient in icu
    p_icu = p.line(source=df_icu, x='x_range_label', y='icu_covid', 
                   line_width=2, color='orange', 
                   legend_label='patient in icu', muted_alpha=0.2)  
    
    # ventilator used
    p_vent = p.line(source=df_icu, x='x_range_label', y='vent_covid', 
                    line_width=2, color='red', 
                    legend_label='patient with vent.', muted_alpha=0.2)  
    
    
    # add hoer tool
    p.add_tools(HoverTool(renderers=[p_bed], 
                          tooltips=bed_tip,
                          mode="vline",
                          anchor='top_right',
                          attachment='right',
                          muted_policy='ignore',
                          formatters={'@date':'datetime'}
                         )
               )
    
    p.add_tools(HoverTool(renderers=[p_icu], 
                          tooltips=icu_tip,
                          mode="vline",
                          attachment='left',
                          muted_policy = 'ignore',
                          formatters={'@date':'datetime'}
                         )
               )
    
    p.add_tools(HoverTool(renderers=[p_vent], 
                          tooltips=vent_tip,
                          mode="vline",
                          attachment='right',
                          muted_policy = 'ignore',
                          formatters={'@date':'datetime'}
                         )
               )
    
    # plot style
    p = style_naked(p)
    
    return p
    

In [11]:
def death_plot(df_death):
    '''plot death stat'''
    
    # prepare extra info
    df_death['x_range_label'] = df_death.date.apply(lambda x:x.strftime('%Y, %b-%d'))    
    
    # x range label
    x_range_label = df_death.x_range_label.to_list()
    
    
    # hover tips
    death_tip = [('date', '@date{%Y, %b-%d}'), 
                 ('death case', '@deaths_new{0,0}'), 
                ]
        
    # initiate figure
    p = figure(plot_width=600, plot_height=300, x_range=x_range_label)
    
    # deaths
    p.vbar(source=df_death, x='x_range_label', top='deaths_new', 
           width=0.5, color='black', legend_label='death case', 
           muted_alpha=0.2)
    
    # add hoer tool
    p.add_tools(HoverTool(tooltips=death_tip,
                          mode="vline",
                          anchor='center',
                          attachment='left',
                          muted_policy='ignore',
                          formatters={'@date':'datetime'}
                         )
               )
        
    # plot style
    p = style_naked(p)
    
    return p

    

In [12]:
def calc_ratio(df, list_ref, col_ref):
    '''calculate of list of ratio'''
    
    for i, (ratio_name, item_name) in enumerate (list_ref):
            
        df[ratio_name] = df[[item_name]].values/df[[col_ref]].values
                    
    return df

In [13]:
def vax_status_plot(df_pop):
    '''plot population vax status'''
    
    # preparation
    df_pop["vax_queue"] = df_pop['total_reg_vax'] - df_pop['dose2_cumul'] - df_pop['dose1_only_cumul']
    df_pop['unreg_vax_adult'] = df_pop['pop_18_above'] - df_pop['total_reg_vax']
    
    # ratio
    item_ratio_pop =  [('dose2_ratio_pop', 'dose2_cumul'),
                       ('dose1_ratio_pop', 'dose1_only_cumul'), 
                       ('queue_ratio_pop', 'vax_queue'),
                       ('unreg_ratio_pop', 'unreg_vax_adult'), 
                       ('youth_ratio_pop', 'pop_below_18'), 
                       ('adult_ratio_pop', 'pop_18_above')
                      ]

    item_ratio_adult = [('dose2_ratio_adult', 'dose2_cumul'),
                        ('dose1_ratio_adult', 'dose1_only_cumul'), 
                        ('queue_ratio_adult', 'vax_queue'),
                        ('unreg_ratio_adult', 'unreg_vax_adult'), 
                       ]
    
    df_pop = calc_ratio(df_pop, item_ratio_pop, 'pop')
    df_pop = calc_ratio(df_pop, item_ratio_adult, 'pop_18_above')
    
    # y range label
    y_range_label = df_pop.state.to_list()    
    
    # hover tips
    youth_tip = [('state', '@state'), 
                 ('pop. below 18', '@pop_below_18{0,0}'),
                 ('over pop.', '@youth_ratio_pop{0.00%}'),
                 ('total pop.', '@pop{0,0}')
                ]
    
    dose2_tip = [('state', '@state'), 
                 ('dose 2', '@dose2_cumul{0,0}'),
                 ('over pop', '@dose2_ratio_pop{0.00%}'),
                 ('over adult pop.', '@dose2_ratio_adult{0.00%}')
                ]
    
    dose1_tip = [('state', '@state'), 
                 ('dose 1', '@dose1_only_cumul{0,0}'),
                 ('over pop', '@dose1_ratio_pop{0.00%}'),
                 ('over adult pop.', '@dose1_ratio_adult{0.00%}')
                ]
    
    queue_tip = [('state', '@state'), 
                 ('vax queuing', '@vax_queue{0,0}'),
                 ('over pop', '@queue_ratio_pop{0.00%}'),
                 ('over adult pop.', '@queue_ratio_adult{0.00%}')
                ]
    
    unreg_tip = [('state', '@state'), 
                 ('unregister adult', '@unreg_vax_adult{0,0}'),
                 ('over pop', '@unreg_ratio_pop{0.00%}'),
                 ('over adult pop.', '@unreg_ratio_adult{0.00%}')
                ]
        
    # initiate figure
    p = figure(plot_width=1200, plot_height=200, y_range=y_range_label, name='vax_status')
           
    # dose 2 completed
    p_dose2 = p.hbar(source=df_pop, y='state', 
                     right='dose2_cumul', left=0, height=0.3,
                     color='#013220', legend_label='dose 2', 
                     muted_alpha=0.2)
    
    # dose 1 completed
    p_dose1 = p.hbar(source=df_pop, y='state', 
                     right=stack('dose2_cumul','dose1_only_cumul'), 
                     left="dose2_cumul", height=0.3,
                     color='green', legend_label='dose 1 only', 
                     muted_alpha=0.2)
    
    # remaining adult already vax-registered
    p_queue = p.hbar(source=df_pop, y='state', 
                     right=stack('dose2_cumul','dose1_only_cumul','vax_queue'), 
                     left=stack('dose2_cumul', 'dose1_only_cumul'), height=0.3,
                     color='#90ee90', legend_label='vax. queuing', 
                     muted_alpha=0.2)
    
    # unregister adult of vax-registered
    p_unreg = p.hbar(source=df_pop, y='state', 
                     right=stack('dose2_cumul','dose1_only_cumul','vax_queue', 'unreg_vax_adult'), 
                     left=stack('dose2_cumul', 'dose1_only_cumul', 'vax_queue'), height=0.3,
                     color='yellow', legend_label='unregister adult', 
                     muted_alpha=0.2)
    
    # population below 18
    p_youth = p.hbar(source=df_pop, y='state', 
                     right=stack('dose2_cumul','dose1_only_cumul','vax_queue', 'unreg_vax_adult', 'vax_queue'), 
                     left=stack('dose2_cumul', 'dose1_only_cumul', 'vax_queue', 'unreg_vax_adult'), height=0.3,
                     color='grey', legend_label='population below 18', 
                     muted_alpha=0.2)
    
    # add hoer tool
    p.add_tools(HoverTool(renderers = [p_youth],
                          tooltips=youth_tip,
                          mode="vline",
                          anchor='center',
                          attachment='left',
                          muted_policy='ignore'
                         )
               )
    
    p.add_tools(HoverTool(renderers = [p_dose2],
                          tooltips=dose2_tip,
                          mode="vline",
                          anchor='center',
                          attachment='left',
                          muted_policy='ignore'
                         )
               )
    
    p.add_tools(HoverTool(renderers = [p_dose1],
                          tooltips=dose1_tip,
                          mode="vline",
                          anchor='center',
                          attachment='left',
                          muted_policy='ignore'
                         )
               )
    
    p.add_tools(HoverTool(renderers = [p_unreg],
                          tooltips=unreg_tip,
                          mode="vline",
                          anchor='center',
                          attachment='left',
                          muted_policy='ignore'
                         )
               )
    
    p.add_tools(HoverTool(renderers = [p_queue],
                          tooltips=queue_tip,
                          mode="vline",
                          anchor='center',
                          attachment='left',
                          muted_policy='ignore'
                         )
               )
    
        
    # plot style
    p = style_naked(p)
    p.legend.location='top_center'
    
    
    
    return p

In [14]:
def vax_estimator(avg_vd1, avg_vd2, d1_bal, d2_bal):
    'forecasting the vax progress'
    
    # remaining days to complete dose 1 for all adults
    n_d1 = np.round(d1_bal/avg_vd1, 0)
    
    # remaining days to complete dose 2 for all adults
    n_d2 = np.round(n_d1 + ((d2_bal-(n_d1*avg_vd2))/(avg_vd1+avg_vd2)), 0)
    
    return n_d1, n_d2

In [15]:
# layout plots

In [16]:
def plot_dashboard(doc_plot):
    
    def update(attr, old, new):
        # ---plot---
        # vax status
        new_vax_df_state = vax_df[(vax_df['state']==new) & (vax_df['date']>=past_90_days)] 
        new_checkin_df_state = checkin_df[(checkin_df['state']==new) & (checkin_df['date']>=past_90_days)] 
        new_p_vax=vax_plot(new_vax_df_state, new_checkin_df_state)
        
        # postive cases status
        new_cases_df_state = cases_df[(cases_df['state']==new) & (cases_df['date']>=past_90_days)]
        new_p_case = case_plot(new_cases_df_state)
        show(new_p_case)
        # icu cases status
        new_icu_df_state = icu_df[(icu_df['state']==new) & (icu_df['date']>=past_90_days)]
        new_p_icu = icu_plot(new_icu_df_state)
        show(new_p_icu)
        # death status
        new_death_df_state = death_df[(death_df['state']==new) & (death_df['date']>=past_90_days)]
        new_p_death = death_plot(new_death_df_state)
        show(new_p_death)
        # population vax status
        new_pop_df_state = pop_df[pop_df['state']==new]
        new_p_vax_pop = vax_status_plot(new_pop_df_state)
        show(new_p_vax_pop)
        # forecast
#         avg_vax_dose = vax_df.set_index('date').last('3D').groupby('state')[['dose1_daily', 'dose2_daily']].mean()
        new_avg_vax_dose_state = avg_vax_dose[avg_vax_dose.index==new]
        new_avg_vax_dose1_value = new_avg_vax_dose_state.iat[0,0]
        new_avg_vax_dose2_value = new_avg_vax_dose_state.iat[0,1]
#         pop_dose1_balance = pop_df.set_index('state')['pop_18_above'] - pop_df.set_index('state')['at_least_dose1']
        new_pop_dose1_bal_value = pop_dose1_balance[pop_dose1_balance.index==new].values
#         pop_dose2_balance = pop_df.set_index('state')['pop_18_above'] - pop_df.set_index('state')['dose2_cumul']
        new_pop_dose2_bal_value = pop_dose2_balance[pop_dose2_balance.index==new].values

        new_day_dose1_bal, new_day_dose2_bal = vax_estimator(new_avg_vax_dose1_value, new_avg_vax_dose2_value, 
                                                             new_pop_dose1_bal_value, new_pop_dose2_bal_value)
        print(new_day_dose1_bal, new_day_dose2_bal)
        new_day_d1_bal = Div(text="Estimated Day To Complete Dose 1 On Adult Population, = %s day(s)" %str(int(new_day_dose1_bal[0])))
        new_day_d2_bal = Div(text="Estimated Day To Complete Dose 2 on Adult Population, = %s day(s)" %str(int(new_day_dose2_bal[0])))


        new_p_layout = layout([[state_sel, 
                           [[Spacer(width=50), new_day_d1_bal, Spacer(width=10), new_day_d2_bal],
                            [new_p_vax_pop],
                            [new_p_vax, Spacer(width=10), new_p_icu],
                            [new_p_case, Spacer(width=10), new_p_death]
                           ]
                          ]])

        p_layout.children=new_p_layout.children

    # # list of state, inlcude name of Malaysia
    list_state = pop_df.state.to_list()

    # selection of states or Malaysia
    state_sel = Select(value=list_state[0], options=list_state, min_width=100, width_policy='auto', default_size=100)
    state_sel.on_change('value', update)

    # ---plot---
    # vax status
    vax_df_state = vax_df[(vax_df['state']==state_sel.value) & (vax_df['date']>=past_90_days)] 
    checkin_df_state = checkin_df[(checkin_df['state']==state_sel.value) & (checkin_df['date']>=past_90_days)] 
    p_vax=vax_plot(vax_df_state, checkin_df_state)

    # postive cases status
    cases_df_state = cases_df[(cases_df['state']==state_sel.value) & (cases_df['date']>=past_90_days)]
    p_case = case_plot(cases_df_state)

    # icu cases status
    icu_df_state = icu_df[(icu_df['state']==state_sel.value) & (icu_df['date']>=past_90_days)]
    p_icu = icu_plot(icu_df_state)

    # death status
    death_df_state = death_df[(death_df['state']==state_sel.value) & (death_df['date']>=past_90_days)]
    p_death = death_plot(death_df_state)

    # population vax status
    pop_df_state = pop_df[pop_df['state']==state_sel.value]
    p_vax_pop = vax_status_plot(pop_df_state)

    # forecast
    # general usage
    avg_vax_dose = vax_df.set_index('date').last('3D').groupby('state')[['dose1_daily', 'dose2_daily']].mean()
    pop_dose1_balance = pop_df.set_index('state')['pop_18_above'] - pop_df.set_index('state')['at_least_dose1']
    pop_dose2_balance = pop_df.set_index('state')['pop_18_above'] - pop_df.set_index('state')['dose2_cumul']
    
    # for default value selection state = Malaysia
    avg_vax_dose_state = avg_vax_dose[avg_vax_dose.index==state_sel.value]
    avg_vax_dose1_value = avg_vax_dose_state.iat[0,0]
    avg_vax_dose2_value = avg_vax_dose_state.iat[0,1]    
    pop_dose1_bal_value = pop_dose1_balance[pop_dose1_balance.index==state_sel.value].values    
    pop_dose2_bal_value = pop_dose2_balance[pop_dose2_balance.index==state_sel.value].values
    
    # get forecasted day for dose1 and dose2 to complete
    day_dose1_bal, day_dose2_bal = vax_estimator(avg_vax_dose1_value, avg_vax_dose2_value, 
                                                 pop_dose1_bal_value, pop_dose2_bal_value)

    day_d1_bal = Div(text="Estimated Day To Complete Dose 1 On Adult Population, = %s day(s)" %str(int(day_dose1_bal[0])))
    day_d2_bal = Div(text="Estimated Day To Complete Dose 2 on Adult Population, = %s day(s)" %str(int(day_dose2_bal[0])))


    p_layout = layout([[state_sel, 
                       [[Spacer(width=50), day_d1_bal, Spacer(width=10), day_d2_bal],
                        [p_vax_pop],
                        [p_vax, Spacer(width=10), p_icu],
                        [p_case, Spacer(width=10), p_death]
                       ]
                      ]])

    doc_plot.add_root(p_layout)
    
handler = FunctionHandler(plot_dashboard)
mycovid19_app = Application(handler)
    
    # as model
#     model = pn.panel(p_layout) #.get_root()
#     model.show()

    # # Put the tabs in the current document for display
    # curdoc().add_root(model)

    # # the doc title
    # curdoc().title = "Malaysia Covid19 - By PointzMatter"

    # # the chart theme
    # curdoc().theme = 'contrast'

    # # show(p_layout)


In [17]:
show(mycovid19_app)

In [18]:
# def update(attr, old, new):
#     # ---plot---
#     # vax status
#     vax_df_state = vax_df[(vax_df['state']==new) & (vax_df['date']>=past_90_days)] 
#     checkin_df_state = checkin_df[(checkin_df['state']==new) & (checkin_df['date']>=past_90_days)] 
#     p_vax=vax_plot(vax_df_state, checkin_df_state)
    
#     # postive cases status
#     cases_df_state = cases_df[(cases_df['state']==new) & (cases_df['date']>=past_90_days)]
#     p_case = case_plot(cases_df_state)

#     # icu cases status
#     icu_df_state = icu_df[(icu_df['state']==new) & (icu_df['date']>=past_90_days)]
#     p_icu = icu_plot(icu_df_state)

#     # death status
#     death_df_state = death_df[(death_df['state']==new) & (death_df['date']>=past_90_days)]
#     p_death = death_plot(death_df_state)

#     # population vax status
#     pop_df_state = pop_df[pop_df['state']==new]
#     p_vax_pop = vax_status_plot(pop_df_state)

#     # forecast
#     avg_vax_dose = vax_df.set_index('date').last('3D').groupby('state')[['dose1_daily', 'dose2_daily']].mean()
#     avg_vax_dose = avg_vax_dose[avg_vax_dose.index==new]
#     avg_vax_dose1 = avg_vax_dose.iat[0,0]
#     avg_vax_dose2 = avg_vax_dose.iat[0,1]
#     pop_dose1_balance = pop_df.set_index('state')['pop_18_above'] - pop_df.set_index('state')['at_least_dose1']
#     pop_dose1_balance = pop_dose1_balance[pop_dose1_balance.index==state_sel.value].values
#     pop_dose2_balance = pop_df.set_index('state')['pop_18_above'] - pop_df.set_index('state')['dose2_cumul']
#     pop_dose2_balance = pop_dose2_balance[pop_dose2_balance.index==state_sel.value].values

#     day_dose1_bal, day_dose2_bal = vax_estimator(avg_vax_dose1, avg_vax_dose2, 
#                                                  pop_dose1_balance, pop_dose2_balance)

#     day_d1_bal = Div(text="Estimated Day To Complete Dose 1 On Adult Population, = %s day(s)" %str(int(day_dose1_bal[0])))
#     day_d2_bal = Div(text="Estimated Day To Complete Dose 2 on Adult Population, = %s day(s)" %str(int(day_dose2_bal[0])))


#     p_layout = layout([[state_sel, 
#                        [[Spacer(width=50), day_d1_bal, Spacer(width=10), day_d2_bal],
#                         [p_vax_pop],
#                         [p_vax, Spacer(width=10), p_icu],
#                         [p_case, Spacer(width=10), p_death]
#                        ]
#                       ]])

#     p_layout.children=p_layout.children

# # # list of state, inlcude name of Malaysia
# list_state = pop_df.state.to_list()

# # selection of states or Malaysia
# state_sel = Select(value=list_state[0], options=list_state, min_width=100, width_policy='auto', default_size=100)
# state_sel.on_change('value', update)

# # ---plot---
# # vax status
# vax_df_state = vax_df[(vax_df['state']==state_sel.value) & (vax_df['date']>=past_90_days)] 
# checkin_df_state = checkin_df[(checkin_df['state']==state_sel.value) & (checkin_df['date']>=past_90_days)] 
# p_vax=vax_plot(vax_df_state, checkin_df_state)

# # postive cases status
# cases_df_state = cases_df[(cases_df['state']==state_sel.value) & (cases_df['date']>=past_90_days)]
# p_case = case_plot(cases_df_state)

# # icu cases status
# icu_df_state = icu_df[(icu_df['state']==state_sel.value) & (icu_df['date']>=past_90_days)]
# p_icu = icu_plot(icu_df_state)

# # death status
# death_df_state = death_df[(death_df['state']==state_sel.value) & (death_df['date']>=past_90_days)]
# p_death = death_plot(death_df_state)

# # population vax status
# pop_df_state = pop_df[pop_df['state']==state_sel.value]
# p_vax_pop = vax_status_plot(pop_df_state)

# # forecast
# avg_vax_dose = vax_df.set_index('date').last('3D').groupby('state')[['dose1_daily', 'dose2_daily']].mean()
# avg_vax_dose = avg_vax_dose[avg_vax_dose.index==state_sel.value]
# avg_vax_dose1 = avg_vax_dose.iat[0,0]
# avg_vax_dose2 = avg_vax_dose.iat[0,1]
# pop_dose1_balance = pop_df.set_index('state')['pop_18_above'] - pop_df.set_index('state')['at_least_dose1']
# pop_dose1_balance = pop_dose1_balance[pop_dose1_balance.index==state_sel.value].values
# pop_dose2_balance = pop_df.set_index('state')['pop_18_above'] - pop_df.set_index('state')['dose2_cumul']
# pop_dose2_balance = pop_dose2_balance[pop_dose2_balance.index==state_sel.value].values

# day_dose1_bal, day_dose2_bal = vax_estimator(avg_vax_dose1, avg_vax_dose2, 
#                                              pop_dose1_balance, pop_dose2_balance)

# day_d1_bal = Div(text="Estimated Day To Complete Dose 1 On Adult Population, = %s day(s)" %str(int(day_dose1_bal[0])))
# day_d2_bal = Div(text="Estimated Day To Complete Dose 2 on Adult Population, = %s day(s)" %str(int(day_dose2_bal[0])))


# p_layout = layout([[state_sel, 
#                    [[Spacer(width=50), day_d1_bal, Spacer(width=10), day_d2_bal],
#                     [p_vax_pop],
#                     [p_vax, Spacer(width=10), p_icu],
#                     [p_case, Spacer(width=10), p_death]
#                    ]
#                   ]])

# # as model
# model = pn.panel(p_layout) #.get_root()
# model.show()

# # # Put the tabs in the current document for display
# # curdoc().add_root(model)

# # # the doc title
# # curdoc().title = "Malaysia Covid19 - By PointzMatter"

# # # the chart theme
# # curdoc().theme = 'contrast'

# # # show(p_layout)


In [19]:
# cluster_df.state.str.split(',')

In [20]:
# # cluster_df[cluster_df.state.str.contains(r'.Kuala Lumpur')]
# cluster_df_90d = cluster_df[cluster_df['date_announced']>=past_90_days]
# cluster_cases_sum = cluster_df_90d.groupby('category').sum()
# cluster_cases_count = cluster_df_90d.groupby('category')['cluster'].count()

In [21]:
# cluster_cases_count.sort_values(ascending=False).plot.bar()

In [22]:
# cluster_cases_sum.sort_values('cases_total', ascending=False)\
# .reset_index().plot.bar(x='category', y='cases_total')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py